In [1]:
import cv2
import numpy as np

# The Function for finding all the templates!

In [2]:
def template_finder(image, template, threshold=0.8):
    output, img_gray = image.copy(), image.copy()
    
    if img_gray.shape[2] == 3:
        img_gray = cv2.cvtColor(img_gray, cv2.COLOR_BGR2GRAY)
    
    if template.shape[2] == 3:
        template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
    
    w, h = template.shape[::-1]
    res = cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED)
    loc = np.where( res >= threshold)
    
    for pt in zip(*loc[::-1]):
        cv2.rectangle(output, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)
    
    return output

## Driver Code

In [ ]:
mainCity = cv2.imread('./MasterOpenCV/FoE/Main.png')
aiding = cv2.imread('./MasterOpenCV/FoE/Aid.png')
FT = cv2.imread('./MasterOpenCV/FoE/FT.png')
GB = cv2.imread('./MasterOpenCV/FoE/GB.png')
MPed = cv2.imread('./MasterOpenCV/FoE/Motivated.png')

toBeAided = template_finder(mainCity, aiding)
friendTavern = template_finder(mainCity, FT)
GB = template_finder(mainCity, GB)
Motivated = template_finder(mainCity, MPed, threshold=0.5)

cv2.imshow('Aiding', toBeAided)
cv2.waitKey()
cv2.imshow('Friend Tavern', friendTavern)
cv2.waitKey()
cv2.imshow('Great Buildings', GB)
cv2.waitKey()
cv2.imshow('Motivated', Motivated)
cv2.waitKey()
cv2.destroyAllWindows()